# Mothur Markdown (and other)

This Markdown file lists the workflow used to recreate the work done by Shengyong Mao et al in
### Characterising the bacterial microbiota across the gastrointestinal tracts of dairy cattle: membership and potential function

---
### SRA download

Initially the files needed to be downloaded from the SRA database provided by NCBI. This was done using the SRA toolkit within the shell.

To locate the file and download it using the command
#you will need to navigate to the folder you have the SRA toolkit downloaded to and run these commands there

`$prefetch -v SRR1778214` 

#this command will create a file called ncbi w/in the users directory.

to split the sequences into forward and reverse reads, navigate to your working directory and use the command

`$fastq-dump --split-files (/home/[USER]/ncbi/public/sra/SRR1778214.sra)`

#alternatively, you can use the -outdir (/output directory) argument to direct the output to your designated directory. Using this will save you the hassle of navigating to the output directory
#the --split-files argument should split the file into forward and reverse reads, however, ours did not. We tried the argument --split-3 which is meant to split and tailor the reads to paired and unpaired, but only one file was output, suggesting only forward reads were present or detectable.
#This was rather frustrating due to fact they claimed to use "paired end" sequencing, which would make quality control and sequence analysis more accurate.

### ENA download

We could not figure out if the file had failed to organize the sequences in a way that the SRA toolkit could recognize forward and reverse, or if it simply did not have reverse included.

We also downloaded the file using the European Nucleotide Database provided program, which did not solve our issue.

Conclusion: forward sequences only were uploaded for this particular experiment

---

## Mothur

To download the latest version of mothur (this analysis used 1.39.0 and is provided within the repositroy). please visit (https://www.mothur.org/wiki/Download_mothur) if you would like to download any other files that aren't mentioned in this analysis.

This "download" page gives you access to all files used in this analysis besides the sequences themselves. We used the latest Silva reference files (silva.bacterial.fasta and silva.gold.align), RDP files (trainset_16), and the 13.5 Greengenes reference file. all of these 

Begin by transferring all reference files and the program to a directory containing your fastq file(s)
 
run the program. It should open it's own shell with the prompt ">"

__1.__

initially we needed to convert the fastq file into the fasta file (containing just sequences w/out quality)

`>fastq.info(fastq=SRR1778214.fastq)`

should return:

`Output File Names: 
SRR1778214.fasta
SRR1778214.qual`

then check the sequences

`>summary.seqs(fasta=SRR1778214.fasta)`

output:

__2.__

From the two files generated, command mothur to trim the sequences with lower that 20 for PHRED qulaity score over a sliding window of 10b. 

`>trim.seqs(fasta=SRR.fasta, qfile=SRR.qual, qwindowsize=10, qwindowaverage=20, minlength=426, maxhomop=8, maxambig=0)`
#arguments qfile=specifies the corresponding quality file, qwindowsize set the rolling window size, qwindowaverage sets the quality score cutoff
#notice, I manually changed the name of my fasta file to SRR.fasta. Shorter names are always better if you can maintain important info.

output:

`Output File Names: 
SRR.trim.fasta
SRR.scrap.fasta
SRR.trim.qual
SRR.scrap.qual`

Then check the sequences again

`>summary.seqs(fasta=SRR.trim.fasta)`

output:

notice that nothing changed here. Why is that? We think it may be that they did some quality control before they uploaded the sequences. Not 100% sure here.

__3.__

Now we do something that is normally usd to handle many sequences from different samples with different barcodes. We will condense these different identifiers into "groups". Even though we only have one sample, this command is needed to make a file used downstream. This command is, however, very useful if handling large sample amounts.

`make.group(fasta=SRR.trim.fasta, groups=Rumen_wall)`
#name the group whatever you would like

output:

`Output File Names: SRR.trim.groups`

__4.__

We will now condence the total number of sequences into unique sequences. This means we will take all of the sequences that are >96% similar select one representative sequence for the entire group. All the sequences will be removed except for the representative sequence, and a count will be included in the "names" file as meta data, keeping track of condensed sequences. The command is:

`>unique.seqs(fasta=SRR.trim.fasta)`

output:

`Output File Names: 
SRR.trim.names
SRR.trim.unique.fasta`

#the names file is produced here, and is mentioned above. the fasta file will continue down the pipe.

Make sure to check the sequences now:

`>summary.seqs(fasta=SRR.trim.unique.fasta)`

output:

#you can include the name file like this, to remove the warning:

`>summary.seqs(fasta=SRR.trim.unique.fasta, name=SRR.trim.names)`

#notice that the number of sequences were condensed by half this means that there are 28,420 sequences that were different in this file (<96%)

__5.__

here we will make a file fore downstream use, once again. This file is called the count_table file, and will include information about the sequences such as the number of sequences condensed per OTU (uniqe.seqs). It will also be able to keep groups distinct, if we had any.

`count.seqs(name=SRR.trim.names, group=SRR.trim.groups)`

output:

`Using 4 processors.
It took 0 secs to create a table for 43974 sequences.


Total number of sequences: 43974

Output File Names:
SRR.trim.count_table`

__6.__

Although you can use the pcr.seq command here to shrink the alignment reference file to a more manageable size, for our final analysis we skipped this. the silva.bacterial.fasta file is ok.

here is the command anyways... if you wanted to include it:

`pcr.seqs(fasta=silva.bacterial.fasta, oligos=MyOligos.oligos)`
#the fasta provided is the silva reference file and the oligos file is a file containing my forward primer sequence and reverse primer sequence. these will be mapped to the reference.

We moved straight to aligning the sequences. for this, we used the command:

`align.seqs(fasta=SRR.trim.unique.fasta, reference=silva.bacteria.fasta, processors=4)`
#here we are providing the fasta file to align, providing the reference database (in the final analysis we are using the silva in one final attempt to get it to accurately be classified), and using all 4 of my local processors. check the number of processors your machine has before running this command.

output:

`Output File Names:
SRR.trim.unique.align
SRR.trim.unique.align.report
SRR.trim.unique.flip.accnos`

and check it:

`summary.seqs(fasta=SRR.trim.unique.align, name=SRR.trim.unique.fasta)`

output:


__5.__

Now we will check the fasta file for chimeric sequences and remove them. Luckily, mothur offers the uchime package and is compatible with a silva reference file strictly for removing chimeras. We used this code for processing chimeras:

`chimera.uchime(fasta=SRR.trim.unique.align, dereplicate=t, reference=silva.gold.align, processors=4)`
#this is supplying the fasta file to be checked, and what reference should be used, in this case the silva.gold.align file.
#dereplicate just says that if one group finds the sequences to be chimeric, the rest will too. this saves time and is suggested to be included, but in this case, we are only using one group and therefore it really doesn't do anything.

#additionally, this command makes a bunch of crazy stuff happen on the screen. we can't tell you why, but it is cleared up upon completion.

__8.__ 

Lets get those chimeras out of our fasta file!

`remove.seqs(fasta=SRR.trim.unique.align, accnos=SRR.trim.unique.ref.uchime.accnos)`

this command finds the flagged chimeric sequences in the original fasta and removes them.

then make sure you check that new fasta:

`summary.seqs(fasta=SRR.trim.unique.pick.align)`

output:

__9.__

Finally, we make it to the classification step. Unfortunately, we could not make it work... However, we think it can, provided the correct input. We feel that this software is truly very powerful, being user friendly and compatible with both mac and windows.  

Anyway, onto the code we used for this analysis:

`classify.seqs(fasta=SRR.trim.unique.pick.align, count=SRR.trim.count_table, reference=trainset16_022016.rdp.fasta, taxonomy=trainset16_022016.rdp.tax, processors=4)`

#this command uses the fasta file you want to classify, the count table we generated earlier, the reference form the RDP database, the taxonomy file from the RDP that contains names for each reference, and the number of processors going.
#it is important to note here that the RDP files only classify to genus level. not that it mattered.

output:

`[WARNING]: mothur reversed some your sequences for a better classification.  If you would like to take a closer look, please check SRR.trim.unique.pick.rdp.wang.flip.accnos for the list of the sequences.

It took 313 secs to classify 23629 sequences.


It took 1 secs to create the summary file for 23629 sequences.


Output File Names:
SRR.trim.unique.pick.rdp.wang.taxonomy

SRR.trim.unique.pick.rdp.wang.tax.summary

SRR.trim.unique.pick.rdp.wang.flip.accnos`

if you open the "SRR.trim.unique.pick.rdp.wang.taxonomy" file with Excel, you will see what we mean when we say it did not work.

we are not sure if it is versioning, poor upstream analysis or if it was bad data, but we did learn some about mothur and feel that, in the right hands, it could be very powerful.

---

Sources:

https://www.mothur.org/wiki/Main_Page

https://www.mothur.org/wiki/RDP_reference_files

https://www.mothur.org/wiki/Silva_reference_files

https://www.mothur.org/wiki/Greengenes-formatted_databases

http://blog.mothur.org/2016/07/07/Customization-for-your-region/

https://www.protocols.io/view/week-8-classifying-taxonomy-of-short-reads-with-mo-g7tbznn

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4630781/